# Reshaping

We've now seen some of the benefits of panel data, and how we can take advantage of pandas to manipulate it and get some insights. Sometimes though we'll need to reshape our data to work with it more easily.

In [66]:
import pandas as pd
import numpy as np

# Load the data
df = pd.read_csv("data/sp500_q1_2025.csv")

# Convert 'datadate' to datetime
df.DlyCalDt = pd.to_datetime(df.DlyCalDt)

# Data cleaning as we did previously
df.dropna(inplace=True)

print("Missing data after cleaning", df.isnull().sum().sum())

# Handling duplicates
print("Checking for duplicates, which we forgot to do previously!", df.duplicated().sum())

df.drop_duplicates(inplace=True)


## Pivot

`pivot` helps us reshape *long* panel data into a *wide* data frame. We can use it to have each stock in a separate column and dates in the rows. We can only have one column substituted in as the values of the data frame, so choose carefully.

In [67]:
# Pivot table with closing prices
pivot_df = df.pivot(index='DlyCalDt', columns='SecurityNm', values='DlyClose')

# Notice that we have time series data again!
pivot_df.loc["2025-02"]

In [68]:
# Also notice that our data is now missing values...
print("Pivoting causes some missing values", pivot_df.isnull().sum().sum())

# Let's find our missing values
missing = pivot_df.isnull().sum()
missing[missing > 0]

# McCormick & Co. only has a single missing value - let's forward fill that
pivot_df["MCCORMICK & CO INC; COM V; CONS"] = pivot_df["MCCORMICK & CO INC; COM V; CONS"].ffill()

# Other securities had significant missing data, so let's drop them
# Missing values have changed, so re-determine them and store
missing = pivot_df.isnull().sum()
to_drop = missing[missing > 0].index

# Drop those columns
pivot_df = pivot_df.drop(columns=to_drop)

print("After cleaning missing values", pivot_df.isnull().sum().sum())


# # Or a less precise method
# pivot_df = pivot_df.dropna(axis=1)

We generally favour returns over close prices, as they give us a better picture of relative performance. Because our data frame is only holding close prices, it is straightforward to calculate returns.

In [69]:
# Daily returns
pivot_df.pct_change()

# Mean daily return of each stock
pivot_df.pct_change().mean()

# For log returns, just use the pivoted df in np.log()
np.log(pivot_df).diff()

The other really neat thing we can do with this kind of pivoted dataframe is visualise correlations with ease.

In [70]:
pivot_df.pct_change().corr()

### Exercise: Trading Top Ten

Pivot our panel data, this time using trading volume `DlyVol` for values. Find the max trading volume for each stock and display the top 10.

In [71]:
## YOUR CODE GOES HERE

## Resample

The other kind of reshaping we can do is called *resampling*, which we use to change the frequency of our data. When we resample, we are generally expected to do some aggregation (but we don't have to). Let's resample our pivoted data to get the mean closing price for each month.

In [72]:
pivot_df.resample("ME").mean()

# Without aggregation
pivot_df.resample("ME").asfreq()

We can use `resample()` to help us calculate returns for different periods. When we calculated daily returns we took the last price on the day and the last price on the day before.

For other periods we apply the same thinking. For monthly returns, for example, we take the last price of the month and the last price of the month before. We'll need `last()` to make it work.

In [73]:
# Chaining methods for monthly returns
pivot_df.resample("ME").last().pct_change()

There are many possibilities for resampling, here are a few:

- **W** - Weekly
- **D** - Daily (calendar days)
- **QE** - Quarterly (quarter end)
- **YE** - Annually

### Exercise: Losing Days

Resample your *trading volume* pivot df to calendar days. Do you need to do some cleaning? What do you propose?

In [74]:
## YOUR CODE GOES HERE